# Video Scraping from Links
- This notebook is for when we have collected labels and links already

- Downloading all videos takes a long time, so we will only scrape selected videos

- With this notebook, we can use `download_videos.py` to:
    1. Download all videos for one word from one data source 
        - (for review during the data organisation and cleaning)
    2. Download all videos for one word from all data sources 
        - (for review during the data organisation and cleaning)
    3. Download all videos for a collection of words from all data source 
        - (for creating our raw combined dataset, after we have decided our target words)

In [3]:
import pathlib
import os
import pandas as pd
import requests
import download_videos

In [4]:
# change working directory to the project root directory
current_dir = os.getcwd()
os.chdir(current_dir + '/../../')
# this should be the project root directory
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage'

## *for review during the data organisation and cleaning*

### 1. Download all videos for one word from one data source

Set the data source

In [5]:
# choose from 'INES, 'V-Librasil', 'SignBank' using 1, 2, 3 (on linux paths are case sensitive)
data_source_key = 1
data_source = {1: 'INES', 2: 'V-Librasil', 3: 'SignBank'}[1]
# get metadata csv path from data_source
metadata_csv_path = pathlib.Path(f'data/raw/{data_source}/metadata.csv')
metadata_csv = pd.read_csv(metadata_csv_path)

Set the word

In [14]:
# choose word that exists in the data source metadata csv file
word = 'BABA'

if word not in metadata_csv['label'].values:
    raise ValueError(f"Word '{word}' not found in metadata csv file")

# get video links from metadata csv
word_metadata = metadata_csv[metadata_csv['label'] == word]

if len(video_links) == 0:
    raise ValueError(f"No video links found for word '{word}' in {data_source} data source")

word_metadata

,label,video_url,file_exists,letter,assuntos,acepção,exemplo,exemplo libras,classe gramatical,origem
663,BABA,https://www.ines.gov.br/dicionario-de-libras/p...,False,B,MATÉRIA/SUBSTÂNCIA,Saliva que corre da boca.,A baba do cachorro é horrível.,BABA CACHORRO HORRÍVEL.,SUBSTANTIVO,Nacional


Download the video
- during the review stage, videos will be downloaded to the `data/raw/{data_source}/videos/review` folder
- during the creation of the (raw) combined dataset, videos will be downloaded to the `data/raw/{data_source}/videos/` folder

Set save path

In [11]:
# during the review stage, videos will be downloaded to the `data/raw/{data_source}/videos/review` folder
# during the creation of the raw combined dataset, videos will be downloaded to the `data/raw/(data_source)/videos` folder

# set paths and folders
review_videos_path = pathlib.Path(f'data/raw/{data_source}/videos/review')

if not review_videos_path.exists():
    os.makedirs(review_videos_path)

Download video(s)

### 2. Download all videos for one word from all data sources

## *for creating our combined raw dataset, after we have decided our target words*

### 3. Download all videos for a collection of words from all data sources